In [28]:
import math
import numpy as np
import cv2
import time
import sys, os

import mediapipe as mp
import pandas as pd
import pprint
from threading import Thread
#import multiprocess
from numba import njit


ENABLE = True
SCALE_FACTOR = 0.5

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose.MODEL_COMPLEXITY = 2
keyPoints = None
def vstack_image(images=[]):
    out = images[0].astype(np.uint8)
    for image in images[1:]:
        out = cv2.vconcat((out, image.astype(np.uint8)))
    return out

def hstack_image(images=[]):
    out = images[0].astype(np.uint8)
    for image in images[1:]:
        out = cv2.hconcat((out, image.astype(np.uint8)))
    return out
    
def resize_image(image, fixed=False):
    return cv2.resize(image, (int(image.shape[1] * SCALE_FACTOR), int(image.shape[0] * SCALE_FACTOR)), interpolation=cv2.INTER_AREA)


def drawStencil(imageDimension):
    image = np.zeros(imageDimension)
    image = drawCircles(image, keyPoints, np.arange(33))
    return image
    
def drawCircles(image, keyPoints, circle_points, columns=(4, 6), small=False):
    _kp = keyPoints[:, columns[0]:columns[1]+1].astype("int32")
    for point in circle_points:
        _temp = _kp[point, :]
        if small:
            image = cv2.circle(image, _temp, radius=1, color=(255, 255, 255), thickness=2)     
        else:
            image = cv2.circle(image, _temp, radius=5, color=(0, 0, 255), thickness=-10)
            image = cv2.circle(image, _temp, radius=7, color=(255, 255, 255), thickness=2)   
    return image


def annotateMeasurements(image, measurements):
    for measure in measurements:
        if measure.get("midpoint", None):
            x, y = measure.get("midpoint")
            image = cv2.putText(image, str(measure.get("distance", "null")), (x, y), cv2.FONT_HERSHEY_SIMPLEX,
                                fontScale=1, thickness=2, color=(255, 255, 255), lineType=cv2.LINE_AA)
    return image

def main():
    global keyPoints, imageShape
    cap = cv2.VideoCapture("/Users/calvin/Documents/NUIG/Thesis/Movies/bowing/bowing8.MP4")
    with mp_pose.Pose(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as pose:
        pose.enable_segmentation = False
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("Ignoring emtpty camera frame.")
                break

            image = resize_image(image)
            imageShape = np.array(image.shape)

            if ENABLE:
                image.flags.writeable = False
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                results = pose.process(image)

                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                mp_drawing.draw_landmarks(
                    image,
                    results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

                keyPoints = storeKeyPoints(results, imageShape)
                image = extractKeyPoints(image)
                
            cv2.imshow("stack", cv2.flip(image, 1))
            if cv2.waitKey(1) & (0xFF == 27 | 0xFF == ord("q")):
                cv2.destroyAllWindows()
                break
        cap.release()

        
def extractKeyPoints(image):
    stencil = drawStencil(image.shape)
    sthread1 = CustomThread(drawSingleLines, (stencil, keyPoints, list(mp_pose.POSE_CONNECTIONS)))

    
    sthread2 = CustomThread(drawMap, (stencil.copy(),  0))
    sthread3 = CustomThread(drawMap, (stencil.copy(), 15))
    sthread4 = CustomThread(drawMap, (stencil.copy(), 16))
    sthread5 = CustomThread(drawMap, (stencil.copy(), 33))    
    
    sthread1.start()
    sthread2.start()
    sthread3.start()
    sthread4.start()
    sthread5.start()
    
    sthread1.join()
    stencil1 = sthread1.ret
    vstack2 = vstack_image([image, stencil1])
    
    sthread2.join()
    sthread3.join()
    stencil2 = sthread2.ret
    stencil3 = sthread3.ret
    vstack1 = vstack_image([stencil3, stencil2]) 
    
    sthread4.join()
    sthread5.join()
    stencil4 = sthread4.ret
    stencil5 = sthread5.ret
    vstack3 = vstack_image([stencil4, stencil5])
    
    hstack1 = hstack_image([vstack1, vstack2, vstack3])
    
    return hstack1

In [ ]:
_empty = None

class CustomThread(Thread):
    def __init__(self, func, args):
        Thread.__init__(self)
        self.func = func
        self.args = args
 
    def run(self):
        self.ret = self.func(*self.args)

def storeKeyPoints(results, shape):
    keyPoints = np.zeros((34, 6))
    for idx, lm in enumerate(results.pose_landmarks.landmark):
        keyPoints[idx, 0] = lm.x 
        keyPoints[idx, 1] = lm.y
        keyPoints[idx, 2] = round(lm.z, 3)
        keyPoints[idx, 3] = round(lm.visibility, 3)
    keyPoints[33, 0] = (keyPoints[24, 0] + keyPoints[23, 0]) / 2
    keyPoints[33, 1] = (keyPoints[24, 1] + keyPoints[23, 1]) / 2
    
    keyPoints[:, 4] = keyPoints[:, 0] * shape[1]
    keyPoints[:, 5] = keyPoints[:, 1] * shape[0]
    #print(pd.DataFrame(keyPoints, columns=["x", "y", "z", "v", "xn", "yn"]))
    return keyPoints

   
def drawLinesWithReference(image, _keyPoints, referencePoint, pointList, pointListColumns):
    '''
    referencePoint   -> idx
    pointList        -> [ ... ]
    pointListColumns -> (x_col, y_col)
    '''
    _kp = _keyPoints[pointList, pointListColumns[0]:pointListColumns[1]+1].astype("int32")
    referencePoint = _keyPoints[referencePoint, 4:6].astype("int32")
    for _pt in _kp:
        image = cv2.line(image, referencePoint, _pt, color=(255, 255, 255), thickness=1, lineType=cv2.LINE_AA)
    return image

def find_angleDistance(_keyPoints, referencePoint, pointList, pointName):
    attach = np.ones((34, 4), dtype="int32")
    for point in pointList:
        y2y1 = max(_keyPoints[referencePoint, 5] - _keyPoints[point, 5], 1)
        x2x1 = max(_keyPoints[referencePoint, 4] - _keyPoints[point, 4], 1)
        attach[point, 0] = math.degrees(math.atan(y2y1/x2x1))
        attach[point, 1] = (y2y1**2) + (x2x1**2)              # sqrt this
        attach[point, 2] = (_keyPoints[referencePoint, 4] + _keyPoints[point, 4])
        attach[point, 3] = (_keyPoints[referencePoint, 5] + _keyPoints[point, 5])
    attach[:, 1] = np.sqrt(attach[:, 1])
    attach[:, 2] = attach[:, 2] / 2
    attach[:, 3] = attach[:, 3] / 2
    _keyPoints = np.concatenate([_keyPoints, attach], axis=1)
    return _keyPoints

def drawMap(stencil, kp):
    _keyPoint = find_angleDistance(keyPoints, kp, range(33), str(kp))
    stencilN  = drawLinesWithReference(stencil, _keyPoint, kp, range(33), (4, 5))
    return stencilN


def drawSingleLines(image, _keyPoints, pointList):
    _kp = _keyPoints[:, 4:6].astype("int32")
    for pt1, pt2 in pointList:
        image = cv2.line(image, _kp[pt1, :], _kp[pt2, :], color=(255, 255, 255), thickness=1, lineType=cv2.LINE_AA)
    return image


start = time.perf_counter()
keyPoints = main()
print("Duration   : {}".format(time.perf_counter() - start))

In [ ]:
Ignoring emtpty camera frame.
Duration   : 12.855270709000251

Ignoring emtpty camera frame.
Duration   : 12.859182041000167

Ignoring emtpty camera frame.
Duration   : 12.84361679200083



In [199]:
dict()

TypeError: cannot convert dictionary update sequence element #0 to a sequence